### 얼굴 비디오나 화상캠을 사용해서 어노잉 오렌지 만들기!

- 참고 사이트: 내 얼굴로 어노잉 오렌지 만들기: https://velog.io/@juijeong8324/%EC%BA%90%EA%B8%80%EC%8A%A4%ED%84%B0%EB%94%94-myface-orange

- 만드려면 기본적으로 cv2, imutils, numpy 그리고 dlib이 필요함

    - dlib library: 이미지 처리, 선형대수, 얼굴의 점 색출 및 머신러닝 알고리즘 사용 가능!

    - 위의 라이브러리를 사용하려면 pip install cmake, pip install dlib를 해줘야 하며, visual studio c++이 있어야지 다운이 되는걸 확인함...

In [1]:
import cv2
import dlib 
import numpy as np
from imutils import face_utils, resize

### 오렌지 사진 준비하기

- imread를 통해 orange_img에 저장

- cv2.resize()를 통해 이미지 사이즈 변경(512x512)

In [2]:
orange_img = cv2.imread('orange.png') 
orange_img = cv2.resize(orange_img, dsize=(512,512)) 

### dlib을 통해 얼굴 점 랜드마크 탐지

- dlib 안에 있는 얼굴 영역 탐지 관련 메소드를 초기화 

- 68개 점의 랜드 마크를 탐지

In [3]:
detector = dlib.get_frontal_face_detector() 
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat') 

In [4]:
cap = cv2.VideoCapture('video.mp4') 

In [5]:

while cap.isOpened():
    # read를 통해 img를 읽기
    ret, img = cap.read() 

    # 보낼 프레임이 없다면 종료 
    if not ret: 
        break

    # 얼굴 영역을 인식,  이때 faces에 얼굴 영역 좌표가 담김    
    faces = detector(img) 

    #Orange 이미지를 복사한 것 
    result = orange_img.copy() 

    # 얼굴이 1개 이상이면, 즉 프레임도 한개 이상, 그중에서 첫번째 얼굴만 사용
    if len(faces) > 0: 
        
        # 그중에서 1개(첫 번째)의 얼굴만을 원한다.
        face = faces[0]  

        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        
        # img에서 해당 face만 crop(잘라내기)해서 face_img에 저장
        face_img = img[y1:y2, x1:x2].copy() 

        # 랜드마크의 68점을 찾는다
        shape = predictor(img, face) 
        
        # 원래 dlib 형태인 shape -> numpy 형태로 변환.
        shape = face_utils.shape_to_np(shape)  

        for p in shape:
            cv2.circle(face_img, center=(p[0]-x1, p[1]-y1), radius=2, color=255, thickness=-1)

        # ** 눈과 입을 잘라서 오렌지 파일에 붙이는 코드 **
        # eyes
        le_x1 = shape[36, 0] # **왼쪽 눈을 자르기 위해 x좌표 36번 39번, y좌표 37번 41번 인덱스가 필요**
        le_y1 = shape[37, 1]
        le_x2 = shape[39, 0]
        le_y2 = shape[41, 1]
        le_margin = int((le_x2 - le_x1) * 0.18) # 너무 가깝게 자르면 안 되니까 margin을 준다 

        re_x1 = shape[42, 0] # 오른쪽 눈
        re_y1 = shape[43, 1]
        re_x2 = shape[45, 0]
        re_y2 = shape[47, 1]
        re_margin = int((re_x2 - re_x1) * 0.18)

        # 여기서 왼쪽 눈과 오른쪽 눈에 margin을 준 값을 crop한다. 
        left_eye_img = img[le_y1-le_margin:le_y2+le_margin, le_x1-le_margin:le_x2+le_margin].copy() 
        right_eye_img = img[re_y1-re_margin:re_y2+re_margin, re_x1-re_margin:re_x2+re_margin].copy()

        left_eye_img = resize(left_eye_img, width=100) # 가로를 100으로 resize
        right_eye_img = resize(right_eye_img, width=100) 

        # ** opencv의 seamlessClone **
        
        # 왼쪽 눈 합성  
        result = cv2.seamlessClone( 
            left_eye_img,
            result, #result(오렌지 이미지를 복사한 것)에 합성 하는 것 
            np.full(left_eye_img.shape[:2], 255, left_eye_img.dtype),
            (100, 200),
            cv2.MIXED_CLONE
        )

        # 오른쪽 눈 합성 
        result = cv2.seamlessClone( 
            right_eye_img,
            result,
            np.full(right_eye_img.shape[:2], 255, right_eye_img.dtype),
            (250, 200),
            cv2.MIXED_CLONE
        )

        # mouth
        mouth_x1 = shape[48, 0]
        mouth_y1 = shape[50, 1]
        mouth_x2 = shape[54, 0]
        mouth_y2 = shape[57, 1]
        mouth_margin = int((mouth_x2 - mouth_x1) * 0.1)

        mouth_img = img[mouth_y1-mouth_margin:mouth_y2+mouth_margin, mouth_x1-mouth_margin:mouth_x2+mouth_margin].copy()

        mouth_img = resize(mouth_img, width=250)
        
        # 입 합성 
        result = cv2.seamlessClone( 
            mouth_img,
            result,
            np.full(mouth_img.shape[:2], 255, mouth_img.dtype),
            (180, 320),
            cv2.MIXED_CLONE
        )

        # cv2.imshow('left', left_eye_img)
        # cv2.imshow('right', right_eye_img)
        # cv2.imshow('mouth', mouth_img)
        # cv2.imshow('face', face_img)

        cv2.imshow('result', result)
        
    # cv2.imshow('img', img)
    if cv2.waitKey(0) == ord('q'):
        break 